In [1]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("./data/ner_dataset.csv",encoding='latin1')

In [4]:
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [8]:
data.shape

(1048575, 4)

In [11]:
data.isna().sum()

Sentence #    1000616
Word                0
POS                 0
Tag                 0
dtype: int64

In [20]:
data[~data["Sentence #"].isna()]

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
24,Sentence: 2,Families,NNS,O
54,Sentence: 3,They,PRP,O
68,Sentence: 4,Police,NNS,O
83,Sentence: 5,The,DT,O
...,...,...,...,...
1048501,Sentence: 47955,Indian,JJ,B-gpe
1048521,Sentence: 47956,Indian,JJ,B-gpe
1048545,Sentence: 47957,Two,CD,O
1048556,Sentence: 47958,They,PRP,O


In [21]:
data = data.fillna(method="ffill")

In [22]:
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [24]:
words = list(set(data["Word"].values))

In [26]:
n_words = len(words)

In [27]:
n_words

35178

In [32]:
len(data["Word"].values.tolist())

1048575

In [30]:
len(data["Word"].tolist())

1048575

In [33]:
class SentenceGetter(object):
    def __init__(self,data):
        self.data = data
        self.n_sent = 1
        self.empty = False
        
    def get_next(self):
        try:
            s = self.data[self.data["Sentence #"] == "Sentence: {}".format(self.n_sent)]
            self.n_sent +=1
            return s["Word"].values.tolist(),s["POS"].values.tolist(),s["Tag"].values.tolist()
        except:
            self.empty =True
            return None,None,None

In [34]:
getter = SentenceGetter(data)

In [36]:
sent,pos,tag = getter.get_next()

In [37]:
sent

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country',
 '.']

In [38]:
pos

['NNS',
 'IN',
 'NNS',
 'VBP',
 'VBN',
 'IN',
 'NNP',
 'TO',
 'VB',
 'DT',
 'NN',
 'IN',
 'NNP',
 'CC',
 'VB',
 'DT',
 'NN',
 'IN',
 'JJ',
 'NNS',
 'IN',
 'DT',
 'NN',
 '.']

In [39]:
tag

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [40]:
from sklearn.base import BaseEstimator,TransformerMixin

In [ ]:
class MemoryTagger(BaseEstimator,TransformerMixin):
    def fit(self,X,y):
        voc ={}
        slef.tags = []
        
        for w,t in zip(X,y):
            if t not in self.tags:
                self.tags.append(t)
            if w not in voc:
                voc[w] = {}
            if t not in voc[w]:
                